# embedding the haunting admonishing agents.

There are two sources of an agent sending a message to the user:
1. **User-Initiated Interaction**: The user actively engages with the agent, prompting it to respond or provide information. This can occur through direct queries, commands, or requests for assistance.
2. **Event-Driven Interaction**: The agent autonomously sends messages based on specific triggers or events. This can include notifications, alerts, reminders, or updates that the agent deems relevant to the user without an explicit prompt. For example, when the next day does not have a planning session on the calendar, the agent might send a reminder to the user to schedule one. So in general this mechanism relies on a cached representation of the user's calendar (which can be kept up to date through webhooks) and a check whether certain conditions are met (e.g., no planning session scheduled for the next day) to decide when to send messages proactively.
In all cases, the message is planned in the APschedular module relative to the event time and contains what action to take, what the user can do to prevent being admonished, how the user should be admonished, and the followup pattern in case it gets ignored.

This admonishing agent is really primeraly a tool agent, mostly deterministic, and invoked by other agents.
so its api is really meant as an mcp tools explaining to other agents how and when to use it.

## One-loop AutoGen + APScheduler wiring (in code)

The runtime and scheduler share the same asyncio loop. APScheduler jobs call back into
AutoGen by sending `FollowUpDue` messages. The pieces are:

- `UserChannelAgent`: the only boundary for human I/O
- `HauntingInterventionHandler`: taps `send_message` to schedule/cancel follow-ups
- `HauntingService`: owns APScheduler + in-memory follow-up state
- `HauntingAgent`: receives `FollowUpDue` and decides what to send

### Minimal usage

```python
from datetime import timedelta
from autogen_core import AgentId
from fateforger.haunt.messages import FollowUpSpec, UserFacingMessage

# Any agent sending to the user goes through UserChannelAgent.
await runtime.send_message(
    UserFacingMessage(
        content="Want me to book that planning session?",
        followup=FollowUpSpec(should_schedule=True, after=timedelta(minutes=10)),
        task_id="planning-session",
    ),
    recipient=AgentId("user_channel", key=str(topic_id)),
)
```

### Cancel-on-reply behavior

When the human replies, forward the inbound message using sender
`AgentId("user_channel", key=...)` and the same topic/task. The intervention handler
will cancel any pending follow-ups for that topic/task automatically.

## Tool-agent integration + DB-backed settings

The admonisher exposes deterministic tools via a ToolAgent (`haunter_tools`).
Settings live in the `admonishment_settings` table, so a review agent can read
or update them without touching the scheduler internals.

### Settings lifecycle

- `set_admonishment_settings(...)` writes preferences (delay, escalation, max attempts).
- `get_admonishment_settings(...)` fetches current preferences.
- `schedule_followup(...)` applies stored defaults when fields are omitted.

### Direct tool call example

```python
import json
from autogen_core import AgentId
from autogen_core import FunctionCall

# Example: update settings for a user
await runtime.send_message(
    FunctionCall(
        name="set_admonishment_settings",
        arguments=json.dumps(
            {
                "user_id": "U123",
                "default_delay_minutes": 15,
                "max_attempts": 3,
                "escalation": "firm",
            }
        ),
        id="settings-1",
    ),
    recipient=AgentId("haunter_tools", key="default"),
)

# Example: schedule a follow-up with defaults pulled from DB
await runtime.send_message(
    FunctionCall(
        name="schedule_followup",
        arguments=json.dumps(
            {
                "message_id": "msg-123",
                "content": "Want me to book that planning session?",
                "user_id": "U123",
                "topic_id": str(topic_id),
            }
        ),
        id="followup-1",
    ),
    recipient=AgentId("haunter_tools", key="default"),
)
```

### Reminder: include user scope on outgoing messages

When using `UserFacingMessage`, pass `user_id` (and optionally `channel_id`) so
stored settings can be applied during scheduling.
